In [ ]:
import streamlit as st
import joblib
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Load model dan vectorizer
model = joblib.load("svm_model_sas.pkl")  # Pastikan model SVM sudah dilatih
vectorizer = joblib.load("tfidf_vectorizer_sas.pkl")  # Pastikan vectorizer sudah dilatih

# Kamus untuk mengganti slang words
slang_dict = {
    "gue": "aku", "gw": "aku", "gua": "aku", "ga": "tidak", "gak": "tidak", "tak": "tidak",
    "yg": "yang", "bgt": "banget", "aja": "saja", "dah": "sudah", "tp": "tapi", "sampe": "sampai",
    "kalo": "kalau"
}

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk preprocessing teks
def preprocess(text):
    # Mengubah ke huruf kecil
    text = text.lower()

    # Menghapus URL, mention (@username), dan karakter selain huruf dan spasi
    text = re.sub(r"http\S+|www.\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # Mengganti slang dengan kata baku
    tokens = text.split()
    tokens = [slang_dict.get(token, token) for token in tokens]

    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]

    # Menggabungkan kembali menjadi satu teks
    return ' '.join(tokens)

# Mapping label ke hasil prediksi
label_map = {0: "Normal", 1: "Kecemasan/Depresi"}

# Menampilkan judul aplikasi
st.title("Prediksi Kesehatan Mental dari Teks")
st.write("Masukkan teks di bawah ini dan sistem akan memprediksi apakah termasuk normal atau mengalami kecemasan/depresi.")

# Input teks dari pengguna
user_input = st.text_area("Teks masukan", height=150)

# Proses ketika tombol prediksi ditekan
if st.button("Prediksi"):
    if user_input.strip() == "":
        st.warning("Masukkan teks terlebih dahulu.")
    else:
        # Melakukan preprocessing pada input
        clean_input = preprocess(user_input)

        # Transformasi teks menjadi vektor menggunakan TF-IDF
        vector = vectorizer.transform([clean_input])

        # Melakukan prediksi dengan model SVM
        prediction = model.predict(vector)[0]

        # Menampilkan hasil prediksi
        st.success(f"Hasil prediksi: **{label_map[prediction]}**")

# Menambahkan tulisan kecil di bawah aplikasi
st.markdown("<p style='font-size: 12px; color: gray;'>*tidak menggantikan peran psikolog</p>", unsafe_allow_html=True)